In [1]:
import pandas as pd
from data.netmarble.code.utils.patent_data_utils import load_pickle
from data.netmarble.code.utils.patent_data_utils import save_pickle, load_pickle
from data.netmarble.code.utils.translate import translate_text
from data.netmarble.code.utils.metric import edit_dist
from data.netmarble.code.glossary_extraction.glossary_utils import cal_edit_dist_ratio, detect_unique_glossary_by_freq_and_special_characters, filter_glossary_for_gold
from tqdm import tqdm
from googletrans import Translator
import pandas as pd
import re

In [2]:
pdf = pd.read_csv("./data/netmarble/data/preprocessed/patent_add_translated_mt_df.csv")
pdf

,German,English,English_length,German_length,German_number,English_number,data_correct,API_V2_Translated
0,Verbundstützen ermöglichen hohe Belastungen be...,Composite columns permit high loads on small c...,8,7,[],[],True,Verbundspalten ermöglichen hohe Lasten bei kle...
1,Daraus kann auch die ganzzahlige Frequenzablag...,The integer frequency error can thus also be d...,9,8,[],[],True,Der Ganzzahlfrequenzfehler kann daher auch bes...
2,"beruht darauf, dass d. Zeitpkt.",based on the fact that d. Zeitpkt.,7,5,[],[],True,"Basierend auf der Tatsache, dass d.Zeitpkt."
3,"Ziel ist eine funktionssichere, wartungsarme, ...","The aim is a functionally reliable, low-mainte...",9,9,[],[],True,"Das Ziel ist eine funktional zuverlässige, war..."
4,Heim-Automation) andererseits für jeweils unte...,home-automation) for different types of networ...,8,7,[],[],True,Hausautomatisierung) für verschiedene Arten vo...
...,...,...,...,...,...,...,...,...
1995,Sie trägt den Läufer (17) des Elektromotors (1).,The shaft bearings are the only motor bearings.,8,8,"['1', '17']",[],False,Die Wellenlager sind die einzigen Motorlager.
1996,Die Verschweißung wird mittels Schock-Verdicht...,Bonding is obtained by means of shock-compress...,7,6,[],[],True,Die Bindung wird mittels Schockkompression erh...
1997,1) unddamit die strömungstechnisch bedingten V...,1) and hence reduce the losses caused by curre...,9,7,['1'],['1'],True,1) und verringern daher die durch Ströme verur...
1998,Hierbei sind die Flächen usw.,The conductor exclaves are sponge-like.,5,5,[],[],True,Die exklaven der Leiter sind schwammartig.


In [3]:
alignment_data = load_pickle("./data/netmarble/data/preprocessed/word_alignments_datasets.pkl")
print(alignment_data[0])

[['Composite', 'columns', 'permit', 'high', 'loads', 'on', 'small', 'cross-sections.'], ['Verbundstützen', 'ermöglichen', 'hohe', 'Belastungen', 'bei', 'kleinen', 'Querschnitten.'], {'0': '0', '1': '0', '2': '1', '3': '2', '4': '3', '5': '4', '6': '5', '7': '6'}, {'0': '0-1', '1': '2', '2': '3', '3': '4', '4': '5', '5': '6', '6': '7'}]


In [4]:
glossary_data = pd.read_csv("./data/netmarble/data/preprocessed/glossary_candidate.csv")
glossary_data
# edit distance?

,src_glossary,trg_glossary,translated_glossary,edit_dist,edit_dist_ratio
0,Composite columns,Verbundstützen,Zusammengesetzte Spalten,18,1.3
1,permit,ermöglichen,erlauben,6,0.5
2,high,hohe,hoch,2,0.5
3,loads,Belastungen,Ladungen,5,0.5
4,on,bei,An,3,1.0
...,...,...,...,...,...
8586,included,vorgesehen,inbegriffen,8,0.8
8587,for,Ferner,zum,6,1.0
8588,method.,werden.,Methode.,5,0.7
8589,independent,Abstreifrichtung,unabhängig,14,0.9


In [5]:
data = pd.read_csv("./data/netmarble/data/preprocessed/gold_glossary.csv")
data

,src_glossary,trg_glossary,translated_glossary,edit_dist,edit_dist_ratio
0,luminescence conversion dye,Lumineszenzkonversionsfarbstoff,Lumineszenzkonvertierung Farbstoff,7,0.2
1,target degree of modulation,Soll-Modulationsgrad,Zielgrad der Modulation,17,0.8
2,semiconductor laser diode,Halbleiterlaserdiode,Halbleiter -Laserdiode,3,0.2
3,exhaust gas recirculation,Abgasrückführung,Abgasrezirkulation,11,0.7
4,flame distributor plate,Flammenverteilerblech,Flammenverteilerplatte,5,0.2
...,...,...,...,...,...
1118,FSR,FSR-Sensoren,FSR,9,0.8
1119,dew,Taupunktfüler,Tau,10,0.8
1120,Eu,Eu,EU,1,0.5
1121,Xa,Xa.,Xa,1,0.3


In [6]:
(len("Taupunktfüler") - len("Tau")) / len("Taupunktfüler")

0.7692307692307693

In [7]:
len("-Sensoren") / len("FSR-Sensoren")

0.75

In [8]:
data_dir = './data/netmarble/data/preprocessed/'
total_glossary_candidates = load_pickle(f'{data_dir}/combination_glossary.pkl')

In [9]:

translated_glossary_df = pd.read_csv(f'{data_dir}/glossary_candidate.csv')
translated_glossary_df

,src_glossary,trg_glossary,translated_glossary,edit_dist,edit_dist_ratio
0,Composite columns,Verbundstützen,Zusammengesetzte Spalten,18,1.3
1,permit,ermöglichen,erlauben,6,0.5
2,high,hohe,hoch,2,0.5
3,loads,Belastungen,Ladungen,5,0.5
4,on,bei,An,3,1.0
...,...,...,...,...,...
8586,included,vorgesehen,inbegriffen,8,0.8
8587,for,Ferner,zum,6,1.0
8588,method.,werden.,Methode.,5,0.7
8589,independent,Abstreifrichtung,unabhängig,14,0.9


In [10]:
translated_glossary_df['edit_dist'] = translated_glossary_df.apply(lambda x: edit_dist(x['trg_glossary'], x['translated_glossary']), axis=1)

In [11]:
translated_glossary_df['edit_dist']

0       18
1        6
2        2
3        5
4        3
        ..
8586     8
8587     6
8588     5
8589    14
8590    11
Name: edit_dist, Length: 8591, dtype: int64

In [12]:
def cal_edit_dist_ratio(df):
    '''
    :param df:
    :return:
    '''
    df['edit_dist_ratio'] = df['edit_dist'] / df['trg_glossary'].str.len()
    df['edit_dist_ratio'] = df['edit_dist_ratio'].round(1)
    return df


def detect_special_character(text, punc='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'):
    pattern = r'[' + punc + ']+'
    if re.search(pattern, text) is not None or '''\\n''' in text:
        return False
    else:
        return True
    

def detect_unique_glossary_by_freq_and_special_characters(df):
    glossary_freq = df['src_glossary'].value_counts().items()
    one_freq_glossary = []
    for row in glossary_freq:
        g, c = row
        if c == 1 and detect_special_character(g):
            one_freq_glossary.append(g)
    return one_freq_glossary


def detect_glossary_by_edit_distance(df, min_score, max_score, one_freq_glossary):
    return df[(df.edit_dist_ratio > min_score) & (df.edit_dist_ratio < max_score) & (df.src_glossary.isin(one_freq_glossary))]


def filter_glossary_for_gold(df, min_score, max_score):
    '''
    :param df:
    :param min_score:
    :param max_score:
    :return:
    '''
    one_freq_glossary = detect_unique_glossary_by_freq_and_special_characters(df)
    df = detect_glossary_by_edit_distance(df, min_score, max_score, one_freq_glossary)

    return df

In [13]:
translated_glossary_df = cal_edit_dist_ratio(translated_glossary_df)  # Task 1
gold_glossary_df = filter_glossary_for_gold(translated_glossary_df, min_score=0.1, max_score=0.9)  # Task 2
gold_glossary_df

,src_glossary,trg_glossary,translated_glossary,edit_dist,edit_dist_ratio
1,permit,ermöglichen,erlauben,6,0.5
3,loads,Belastungen,Ladungen,5,0.5
10,integer,ganzzahlige,ganze Zahl,6,0.5
11,frequency error,Frequenzablage,Frequenzfehler,6,0.4
32,functionally,"funktionssichere,",funktional,9,0.5
...,...,...,...,...,...
8544,split,geteilte,Teilt,4,0.5
8545,blow mould,Blasform,Blasenform,2,0.2
8554,susceptible,störungsanfällig.,anfällig,9,0.5
8578,and hence,unddamit,und daher,4,0.5


In [14]:
gold_glossary_df.sort_values(by="src_glossary", key=lambda x: x.str.len(), ascending=False, inplace=True)
gold_glossary_df.to_csv(f'{data_dir}/gold_glossary_mine.csv', index=False)

/home/yehoon/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [16]:
pdf.head()

# src: en / trg: de

,German,English,English_length,German_length,German_number,English_number,data_correct,API_V2_Translated
0,Verbundstützen ermöglichen hohe Belastungen be...,Composite columns permit high loads on small c...,8,7,[],[],True,Verbundspalten ermöglichen hohe Lasten bei kle...
1,Daraus kann auch die ganzzahlige Frequenzablag...,The integer frequency error can thus also be d...,9,8,[],[],True,Der Ganzzahlfrequenzfehler kann daher auch bes...
2,"beruht darauf, dass d. Zeitpkt.",based on the fact that d. Zeitpkt.,7,5,[],[],True,"Basierend auf der Tatsache, dass d.Zeitpkt."
3,"Ziel ist eine funktionssichere, wartungsarme, ...","The aim is a functionally reliable, low-mainte...",9,9,[],[],True,"Das Ziel ist eine funktional zuverlässige, war..."
4,Heim-Automation) andererseits für jeweils unte...,home-automation) for different types of networ...,8,7,[],[],True,Hausautomatisierung) für verschiedene Arten vo...


In [41]:
src = pdf.iloc[0, 1]

In [42]:
def preprocessing_src_sentence(src, df):
    preprocessing_glossary_dict = {}
    glossary_num = 1
    for src_glossary, trg_glossary in df[['src_glossary', 'trg_glossary']].values:
        src_glossary = src_glossary.split(' ')
        if isinstance(src, str):
            src = src.split(' ')
        preprocessing_symbol = '{' + f'Magellan{glossary_num}'+ '}'
        if len(src_glossary) > 1:
            for i in range(0, len(src) - len(src_glossary)+1):
                if src[i:i+len(src_glossary)] == src_glossary or ' '.join(src_glossary) in ' '.join(src[i:i+len(src_glossary)]):
                    src = ' '.join(src)
                    src_glossary = ' '.join(src_glossary)
                    src = src.replace(src_glossary, preprocessing_symbol)
                    preprocessing_glossary_dict[preprocessing_symbol] = trg_glossary
                    glossary_num += 1

        else:
            if src_glossary[0] in src:
                src = ' '.join(src)
                src_glossary = ' '.join(src_glossary)
                src = src.replace(src_glossary, preprocessing_symbol)
                preprocessing_glossary_dict[preprocessing_symbol] = trg_glossary
                glossary_num += 1
        if glossary_num > 3:
            break
    if isinstance(src, list):
        src = ' '.join(src)
    return src, preprocessing_glossary_dict     

def postprocessing_trg_sentence(translated_preprocessing_src, preprocessing_glossary_dict):
    for k, v in preprocessing_glossary_dict.items():
        translated_preprocessing_src = translated_preprocessing_src.replace(k, v)
    return translated_preprocessing_src